In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("final_project.ipynb")

In [ ]:
%%capture
import sys
!{sys.executable} -m pip install pycryptodome

In [ ]:
import hashlib
import numpy as np
import math
from utils import *
from operator import mul

# Final Project: Challenges

Welcome to the challenge version of final project! **NOTE: You only need to complete either this challenge project OR the open-ended project found on the website.**

There are **6** challenges of varying difficulty levels:

Easy: <span style="color:green">●</span>

Medium: <span style="color:orange">● ● </span> 

Hard: <span style="color:red">● ● ● </span>


Each is worth a corresponding amount of points (1 for Easy, 2 for Medium, 3 for Hard). To get credit for the project, you to get at least **7** points out of a possible **12**. This means you don't have to do all the challenges, just enough to get over the point minimum. For example, doing 2 Easy, 1 Medium, and 1 Hard Challenge is enough to pass.

Each challenge will have its own local test(s). Passing the local tests means you will get credit for the challenge. **To submit, go to the Gradescope assignment titled 'Final Project', enter in the final answer, and a short explanation for how you solved the challenge.** 

Gradescope link: https://www.gradescope.com/courses/479588/assignments/2816480/

It may be helpful to revisit / use functions from old labs for some of the challenges.

# Easy Challenges

Each challenge is worth 1 point.

### Challenge 1: "One-Time" Pad

Difficulty: Easy <span style="color:green">● </span>

Relevant topics: One-Time Pad

Alice has made the poor decision to 'roll her own crypto' and comes up with the following scheme to encrypt multiple messages:

$$Enc(k, m_i) = k \oplus m_{i-1} \oplus m$$

She reasons that this is secure, since the "key" gets updated each message. If $i=0$ (i.e. the first message), we set $m_{i-1} = 0$.

You have managed to intercept $Enc(k, m_0)$ and $Enc(k, m_1)$.

**Find the value of $m_1$.**

*HINT: You can XOR two int values in Python using ^: (a ^ b) = $a \oplus b$*

In [ ]:
enc_m1 = 230546131670210616415540217481355295762716095558420049068789227666874881
enc_m2 = 229654869193562975897942432218153746972027379121729058108981474607776588

We can use the functions **textToInt** and **intToText** to convert between text and ints:

In [ ]:
intToText(textToInt("I hope you liked Codebreaking!"))

Feel free to use the following cell, or create new cells to code in:

Put your answer here when you've finished.

In [ ]:
challenge_one_solution = ...

Let's see what $m_2$ decrypts to! Note that this expects `challenge_one_solution` to be in integer form.

**Use this text output as your answer in the Gradescope submission.**

In [ ]:
intToText(challenge_one_solution)

In [ ]:
grader.check("q1")

### Challenge 2: Encrypt-and-Sign

Difficulty: Easy <span style="color:green">● </span>

Relevant topics: Digital Signatures

Alice wishes to send a secret message to Bob. She also wants to ensure Bob can trust the message was from her. She decides to send 

$$C = RSAEncrypt(PK_{Bob}, M)$$

and a signature on the ciphertext 

$$S = RSASign(SK_{Alice}, M)$$

Note that their signature scheme does not use hashes, e.g. $$RSASign(d, m) = m^d \mod N$$

**Recover the original message, given the encrypted message and signature.**

In [ ]:
Alice_PK = {
    'e': 65537,
    'N':  253178424041486225941355620869011259958799253442855343934388957566334587454598725796755179284686502997571220897195182327994815211718724458648084072969341
}

In [ ]:
Bob_PK = {
    'e': 65537,
    'N': 10472999659637508893356384437659733790493176101355411988047099256729779819399531793310494966080656413269645993178088186690122973695672719434956613337820653
}

In [ ]:
C = 8548473624092306473411544941006167134884908489415480218197360098557644599185237838489748379166593812101304047635401022678483559718387508151081153432901606
S = 250739514873490315182488322315337403273432833044991466435423038050117779984774221311497628032704404318987776038245167993904567204971119126897757177003035

Feel free to use the following cell, or create new cells to code in:

In [ ]:
# Make sure to write it as an integer
challenge_two_solution = ...

I wonder what the message was?

**Use this text representation as your answer to the Gradescope submission.**

In [ ]:
intToText(challenge_two_solution)

In [ ]:
grader.check("q2")

# Medium Challenges

Each challenge is worth 2 points.

### Challenge 3: 0xDEADBEEF

Difficulty: Medium <span style="color:orange">● ● </span>

Evanbot has decides to send a message to Codabot with AES-CTR. Since both of their courses often use 0xDEADBEEF as a constant, they decide to use 0xDEADBEEF as the IV for all messages.

**Recover the plaintext of the second message, given both encrypted messages and the plaintext of the first message.**


*HINT: The bxor (byte XOR) function in utils.py may be helpful.*

In [ ]:
ciphertext_one = b'iB\xdb\xac\x1c\xec\xceUE\xfc\xdc((A\x14\x13'
ciphertext_two = b'uO\xde\xb3,\xda\xdbxI\xfe\xf1"C!q\x16'

known_plaintext_one = b'hello_from_soda'

Feel free to use the following cell, or create new cells to code in:

You should recover a bytestring with legible English text (and possibly some padding). **Use this bytestring as the answer for Gradescope (and the field below).**

In [ ]:
# The answer should be formatted as a bytestring, like b'hello_from_soda'
challenge_three_solution = ...

In [ ]:
grader.check("q3")

### Challenge 4: Dual EC DRBG Again?

Difficulty: Medium <span style="color:orange">● ● </span>

Relevant Topics: Elliptic Curve Cryptography, PRNGs

You've been tasked to cryptanalyze the Dual EC DRBG pseudorandom number generator. The constants used in the RNG are found below, and the code used for the PRNG is also listed.

As an aspiring cryptanalyst, you've also discovered that $P = 27Q$, where $P$ and $Q$ are the points used as constants.

**Recover the internal state, given access to the first output.**

Here is the code used to generate the RNG outputs. **Note that no bits are truncated like in real Dual EC DRBG.**

In [ ]:
def dual_EC_drbg(curve, modulus, P, Q, seed):
    assert len(curve) == 2
    assert isElliptic(curve, modulus)
    assert pointOnCurve(P, curve, modulus)
    assert pointOnCurve(Q, curve, modulus)
    
    state = seed
        
    def generateBits():
        nonlocal state 
        nonlocal P
        nonlocal Q
                
        newPoint = doubleAndAdd(P, seed, curve, modulus)
                
        r = newPoint[0]
        
        newQ = doubleAndAdd(Q, r, curve, modulus)
                
        output = newQ[0]
                
        state = doubleAndAdd(P, r, curve, modulus)[0]
                
        return output
    return generateBits

In [ ]:
ellipticCurve = [0, 7]
ellipticCurveModulus = 115792089237316195423570985008687907853269984665640564039457584007908834671663 
generatorPoint = (55066263022277343669578718895168534326250603453777594175500187360389116729240,
 32670510020758816978083085130507043184471273380659243275938904335757337482424)
generatorPointOrder =115792089237316195423570985008687907852837564279074904382605163141518161494337

Q = generatorPoint

# P = 27Q
P = (24049875635381557237058143631624836741422505207761609709712554171343558302165,
 22669890352939653242079781319904043788036611953081321775127194249638113810828)

Alice has instantiated the PRNG with some secret seed. Here is the latest output of the PRNG:

In [ ]:
output = 16707928130850354535739926876056172194711802481891336024715258166049082954370

Reminder: your goal is to recover the *current state value* of the PRNG, not the original seed.

Feel free to use the following cell, or create new cells to code in. You may want to revisit the elliptic curve lab and utils.py for helpful functions.

**Your final answer should be an integer. Use this as the Gradescope submission, and for the field below.**

In [ ]:
# Make sure to write this as an integer (i.e. not as a function evaluation)
challenge_four_solution = ...

In [ ]:
grader.check("q4")

# Hard Challenges
Each challenge is worth 3 points.

### Challenge 5:  Broken Generators

Difficulty: Hard <span style="color:red">● ● ●</span>


Diffie-Hellman provides authenticity, but not integrity. We are an attacker interested in learning the final result of a Diffie-Hellman key exchange. We cannot use any MITM attacks, but are allowed to **modify the generator used in the process**. Note that we cannot set the generator to 1 or $p-1$, as the parties will detect and reject this.

**Set the generator some constant value, and write a function to recover the shared secret given the public values of the Diffie-Hellman exchange.**

**You do NOT need to use any fancy discrete-logarithm algorithms. Brute force works if the exploit is written properly.**

*HINT: $\phi(p) = 2 \cdot 3 \cdot 43 \cdot 2013976508064188041825691454837161339$, where $\phi$ represents Euler's totient function.*

*HINT: $\text{ord}(x^a) = \frac{\text{ord(x)}}{\gcd(\text{ord(x)}, a)}$*

In [ ]:
p = 519605939080560514791028395347987625463

In [ ]:
phi_p = 2 * 3 * 43 * 2013976508064188041825691454837161339

**You should copy-paste the values and functions you write below as the Gradescope submission, once you pass the local tests.**

In [ ]:
# Your solution should work with *any* generator, but p will be fixed
def returnNewGenerator(g, p):
    ...

def recover_dh_secret(new_g, p, ga_mod_p, gb_mod_p):
    ...

You can test your code below.

In [ ]:
a = random.randint(2, p-1)
b = random.randint(2, p-1)

g = 2

new_g = returnNewGenerator(2, p)

ga_mod_p = pow(new_g, a, p)
gb_mod_p = pow(new_g, b, p)

assert pow(new_g, a*b, p) == recover_dh_secret(new_g, p, ga_mod_p, gb_mod_p)

In [ ]:
grader.check("q5")

### Challenge 6: In Defense of Limiters

Difficulty: Hard <span style="color:red">● ● ●</span>

Relevant Topics: RSA, Digital Signatures

Servers often use certificate authorities to issues certificates, ensuring a client can trust that their public key is actually associated with the given server. To this end, the certificate authority is implicitly trusted by the client, and has their public key hardcoded into all clients (i.e. cannot be tampered with).

**Write a function to generate a malicious keypair that still passes certificate validation under the same certificate.**

In [ ]:
CERT_AUTH_PUBLIC_KEY = { 'e': 65537,
             'N': 33790420484761320225234266446986435791020053290995177788399417698648848366075439013295931744537889745793682732187585867814285806211190774412138926826806937374931229955338241741978503726324443629746710612128866806815968501932728765477787763877641403710570749219182260822344263730489611164845428107854720086677}

In [ ]:
SERVER_PUBLIC_KEY = {
        'e': 65537,
        'N': 3002132409982411885213797848544015865850589177629726369826041571611243748750797310385521637461408395352464480231701204492274911135865370769445872391320297
}

In [ ]:
SERVER_CERTIFICATE = 29988389308377186804188678153533881382488143264455800104888748751643200264760653733663959655661660673590410341403741512638892536877052972486958518522731823742528059497839582218210357334986778758862385235675013294056340445805228901167929872913694030281755335353233819756297874410596314423870792706573097000563

In [ ]:
def verifyServerCertificate(SERVER_PUBLIC_KEY, CERT_AUTH_PUBLIC_KEY, server_certificate):
    return verifyRSA(CERT_AUTH_PUBLIC_KEY['e'], 
                     server_certificate,
                     CERT_AUTH_PUBLIC_KEY['N'],
                     int(str(SERVER_PUBLIC_KEY['e']) + str(SERVER_PUBLIC_KEY['N'])))

Notice how the current values check out:

In [ ]:
verifyServerCertificate(SERVER_PUBLIC_KEY, CERT_AUTH_PUBLIC_KEY, SERVER_CERTIFICATE)

**Note that your solution need only work for the specific listed values of SERVER_PUBLIC_KEY and SERVER_CERTIFICATE.**

Recall that $N = pq$ for primes $p, q$. Recovering the private key can be done by finding $p, q$, so that is what we will focus on for this challenge.

**Copy-paste the entire cell below (e = ..., p = ..., q = ...) as your answer for Gradescope.**

In [ ]:
e_solution = ...
p_solution = ...
q_solution = ...

In [ ]:
grader.check("q6")

**Congrats on reaching the end! We hope you enjoyed the Codebreaking DeCal and this project!**


***REMINDER: DO NOT SUBMIT THE ZIP LIKE A NORMAL LAB. Fill in the assignment for each challenge with the solution and a short explanation.***

Gradescope link: https://www.gradescope.com/courses/479588/assignments/2816480/

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Once you have generated the zip file, go to the Gradescope page for this assignment to submit.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)